In [3]:
!pip install forex-python

  Using cached forex_python-1.8-py3-none-any.whl.metadata (1.3 kB)
Using cached forex_python-1.8-py3-none-any.whl (8.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.6/74.6 kB 2.5 MB/s eta 0:00:00


In [4]:
import pandas as pd
from forex_python.converter import CurrencyRates
from datetime import datetime, timedelta

def get_historical_currency_prices(base_currency, target_currency, start_date, end_date):
    """
    Fetch historical currency prices for a given base and target currency within a date range.
    
    :param base_currency: The base currency (e.g., 'USD').
    :param target_currency: The target currency (e.g., 'EUR').
    :param start_date: Start date as 'YYYY-MM-DD'.
    :param end_date: End date as 'YYYY-MM-DD'.
    :return: DataFrame with historical exchange rates.
    """
    c = CurrencyRates()
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")

    date_range = pd.date_range(start=start_date, end=end_date)
    rates = []

    for date in date_range:
        try:
            rate = c.get_rate(base_currency, target_currency, date)
            rates.append({'date': date, 'rate': rate})
        except Exception as e:
            print(f"Error fetching rate for {date}: {e}")
            rates.append({'date': date, 'rate': None})
    
    return pd.DataFrame(rates)

# Example Usage
if __name__ == "__main__":
    base_currency = 'USD'
    target_currency = 'EUR'
    start_date = '2022-01-01'
    end_date = '2022-12-31'
    
    data = get_historical_currency_prices(base_currency, target_currency, start_date, end_date)
    print(data)

    # Save to CSV
    data.to_csv("historical_currency_prices.csv", index=False)


Error fetching rate for 2022-01-01 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-02 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-03 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-04 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-05 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-06 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-07 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-08 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-09 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-10 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-11 00:00:00: Expecting value: line 1 column 1 (char 0)
Error fetching rate for 2022-01-12 00:00:00

KeyboardInterrupt: 

In [ ]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Set up API key and endpoint
API_KEY = "c03e9e40-62b6-4f97-a685-bd650f8ebd81"  # Replace with your CoinMarketCap API key
BASE_URL = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical"

# Cryptocurrencies of interest
CRYPTO_SYMBOLS = ["BTC", "ETH", "ADA", "XRP", "DOGE", "SOL", "ALGO"]
START_DATE = "2023-01-01"  # Historical data start date (YYYY-MM-DD)
END_DATE = "2023-12-31"  # Historical data end date (YYYY-MM-DD)

# Function to fetch historical data
def fetch_historical_prices(symbol, start_date, end_date):
    url = f"{BASE_URL}"
    headers = {"X-CMC_PRO_API_KEY": API_KEY}
    params = {
        "symbol": symbol,
        "time_start": start_date,
        "time_end": end_date,
        "convert": "USD",
        "interval": "daily",
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if "data" in data:
            return data["data"]["quotes"]
        else:
            print(f"No data available for {symbol}")
            return []
    else:
        print(f"Failed to fetch data for {symbol}: {response.status_code}")
        return []

# Function to process and save data
def process_and_save_data():
    all_data = []
    for symbol in CRYPTO_SYMBOLS:
        print(f"Fetching historical data for {symbol}...")
        historical_data = fetch_historical_prices(
            symbol,
            datetime.strptime(START_DATE, "%Y-%m-%d").timestamp(),
            datetime.strptime(END_DATE, "%Y-%m-%d").timestamp(),
        )
        for entry in historical_data:
            date = entry["time_open"][:10]  # Extract date (YYYY-MM-DD)
            open_price = entry["quote"]["USD"]["open"]
            high_price = entry["quote"]["USD"]["high"]
            low_price = entry["quote"]["USD"]["low"]
            close_price = entry["quote"]["USD"]["close"]
            volume = entry["quote"]["USD"]["volume"]
            all_data.append([symbol, date, open_price, high_price, low_price, close_price, volume])

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(
        all_data,
        columns=["Symbol", "Date", "Open", "High", "Low", "Close", "Volume"]
    )
    df.to_csv("crypto_historical_prices.csv", index=False)
    print("Data saved to 'crypto_historical_prices.csv'")

# Main execution
if __name__ == "__main__":
    process_and_save_data()


Fetching historical data for BTC...
Failed to fetch data for BTC: 403
Fetching historical data for ETH...
Failed to fetch data for ETH: 403
Fetching historical data for ADA...
Failed to fetch data for ADA: 403
Fetching historical data for XRP...
Failed to fetch data for XRP: 403
Fetching historical data for DOGE...
Failed to fetch data for DOGE: 403
Fetching historical data for SOL...
Failed to fetch data for SOL: 403
Fetching historical data for ALGO...
Failed to fetch data for ALGO: 403
Data saved to 'crypto_historical_prices.csv'


In [8]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def scrape_bitcoin_historical_data():
    # URL for Bitcoin historical data on CoinCodex
    url = "https://coincodex.com/crypto/bitcoin/historical-data/"
    
    # Send GET request to the webpage
    response = requests.get(url)
    
    # Check if the response is successful
    if response.status_code != 200:
        print(f"Failed to fetch the webpage. Status code: {response.status_code}")
        return None
    
    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Find the table with historical data
    table = soup.find("table", class_="table table-striped table-condensed")
    if not table:
        print("Failed to find the historical data table.")
        return None
    
    # Extract table headers
    headers = [th.text.strip() for th in table.find_all("th")]
    
    # Extract table rows
    rows = []
    for tr in table.find_all("tr")[1:]:  # Skip header row
        cells = [td.text.strip() for td in tr.find_all("td")]
        if cells:  # Ignore empty rows
            rows.append(cells)
    
    # Convert to DataFrame
    df = pd.DataFrame(rows, columns=headers)
    
    # Save the DataFrame to a CSV file
    df.to_csv("bitcoin_historical_data.csv", index=False)
    print("Data saved to 'bitcoin_historical_data.csv'")
    
    return df

# Run the scraper
if __name__ == "__main__":
    bitcoin_data = scrape_bitcoin_historical_data()
    if bitcoin_data is not None:
        print(bitcoin_data.head())


Failed to find the historical data table.


In [ ]:
api_key = c03e9e40-62b6-4f97-a685-bd650f8ebd81


In [1]:
import requests
import pandas as pd
from datetime import datetime, timedelta

# Set up API key and endpoint
API_KEY = "c03e9e40-62b6-4f97-a685-bd650f8ebd81"  # Replace with your CoinMarketCap API key
BASE_URL = "https://pro-api.coinmarketcap.com/v1/cryptocurrency/ohlcv/historical"

# Cryptocurrencies of interest
CRYPTO_SYMBOLS = ["BTC", "ETH", "ADA", "XRP", "DOGE", "SOL", "ALGO"]
START_DATE = "2023-01-01"  # Historical data start date (YYYY-MM-DD)
END_DATE = "2023-12-31"  # Historical data end date (YYYY-MM-DD)

# Function to fetch historical data
def fetch_historical_prices(symbol, start_date, end_date):
    url = f"{BASE_URL}"
    headers = {"X-CMC_PRO_API_KEY": API_KEY}
    params = {
        "symbol": symbol,
        "time_start": start_date,
        "time_end": end_date,
        "convert": "USD",
        "interval": "daily",
    }

    response = requests.get(url, headers=headers, params=params)
    if response.status_code == 200:
        data = response.json()
        if "data" in data:
            return data["data"]["quotes"]
        else:
            print(f"No data available for {symbol}")
            return []
    else:
        print(f"Failed to fetch data for {symbol}: {response.status_code}")
        return []

# Function to process and save data
def process_and_save_data():
    all_data = []
    for symbol in CRYPTO_SYMBOLS:
        print(f"Fetching historical data for {symbol}...")
        historical_data = fetch_historical_prices(
            symbol,
            datetime.strptime(START_DATE, "%Y-%m-%d").timestamp(),
            datetime.strptime(END_DATE, "%Y-%m-%d").timestamp(),
        )
        for entry in historical_data:
            date = entry["time_open"][:10]  # Extract date (YYYY-MM-DD)
            open_price = entry["quote"]["USD"]["open"]
            high_price = entry["quote"]["USD"]["high"]
            low_price = entry["quote"]["USD"]["low"]
            close_price = entry["quote"]["USD"]["close"]
            volume = entry["quote"]["USD"]["volume"]
            all_data.append([symbol, date, open_price, high_price, low_price, close_price, volume])

    # Convert to DataFrame and save to CSV
    df = pd.DataFrame(
        all_data,
        columns=["Symbol", "Date", "Open", "High", "Low", "Close", "Volume"]
    )
    df.to_csv("crypto_historical_prices.csv", index=False)
    print("Data saved to 'crypto_historical_prices.csv'")

# Main execution
if __name__ == "__main__":
    process_and_save_data()


Fetching historical data for BTC...
Failed to fetch data for BTC: 403
Fetching historical data for ETH...
Failed to fetch data for ETH: 403
Fetching historical data for ADA...
Failed to fetch data for ADA: 403
Fetching historical data for XRP...
Failed to fetch data for XRP: 403
Fetching historical data for DOGE...
Failed to fetch data for DOGE: 403
Fetching historical data for SOL...
Failed to fetch data for SOL: 403
Fetching historical data for ALGO...
Failed to fetch data for ALGO: 403
Data saved to 'crypto_historical_prices.csv'


In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Lists to store scraped data
scrape_date_list = []
crypto_date_list = []
crypto_name_list = []
crypto_symbol_list = []
crypto_market_cap_list = []
crypto_price_list = []
crypto_circulating_supply_list = []
crypto_voulume_24hr_list = []
crypto_pct_1hr_list = []
crypto_pct_24hr_list = []
crypto_pct_7day_list = []

# Step 1: Scrape available historical dates
def scrape_date():
    url = 'https://coinmarketcap.com/historical/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    a_tags = soup.find_all('a', class_='historical-link cmc-link')
    for tag in a_tags:
        href = tag.get('href')
        scrape_date_list.append(href)

    print(f'There are {len(scrape_date_list)} dates (Sundays) available for scraping from CoinMarketCap historical data.')

# Step 2: Scrape cryptocurrency data for a specific date
def scrape_data(date):
    url = 'https://coinmarketcap.com' + date
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    tr = soup.find_all('tr', attrs={'class': 'cmc-table-row'})
    count = 0
    for row in tr:
        if count == 10:  # Limit to top 10 cryptocurrencies
            break
        count += 1

        try:
            crypto_date = date
        except AttributeError:
            crypto_date = None

        try:
            name_column = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__name'})
            crypto_name = name_column.find('a').text.strip()
        except AttributeError:
            crypto_name = None

        try:
            crypto_symbol = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__symbol'}).text.strip()
        except AttributeError:
            crypto_symbol = None

        try:
            crypto_market_cap = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__market-cap'}).text.strip()
        except AttributeError:
            crypto_market_cap = None

        try:
            crypto_price = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__price'}).text.strip()
        except AttributeError:
            crypto_price = None

        try:
            crypto_circulating_supply = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__circulating-supply'}).text.strip().split(' ')[0]
        except AttributeError:
            crypto_circulating_supply = None

        try:
            crypto_voulume_24hr_td = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__volume-24-h'})
            crypto_voulume_24hr = crypto_voulume_24hr_td.find('a').text.strip()
        except AttributeError:
            crypto_voulume_24hr = None

        try:
            crypto_pct_1hr = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__percent-change-1-h'}).text.strip()
        except AttributeError:
            crypto_pct_1hr = None

        try:
            crypto_pct_24hr = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__percent-change-24-h'}).text.strip()
        except AttributeError:
            crypto_pct_24hr = None

        try:
            crypto_pct_7day = row.find('td', attrs={'class': 'cmc-table__cell--sort-by__percent-change-7-d'}).text.strip()
        except AttributeError:
            crypto_pct_7day = None

        crypto_date_list.append(crypto_date)
        crypto_name_list.append(crypto_name)
        crypto_symbol_list.append(crypto_symbol)
        crypto_market_cap_list.append(crypto_market_cap)
        crypto_price_list.append(crypto_price)
        crypto_circulating_supply_list.append(crypto_circulating_supply)
        crypto_voulume_24hr_list.append(crypto_voulume_24hr)
        crypto_pct_1hr_list.append(crypto_pct_1hr)
        crypto_pct_24hr_list.append(crypto_pct_24hr)
        crypto_pct_7day_list.append(crypto_pct_7day)

# Step 3: Process and save the data
def process_and_save_data():
    df = pd.DataFrame({
        'Date': crypto_date_list,
        'Name': crypto_name_list,
        'Symbol': crypto_symbol_list,
        'Market Cap': crypto_market_cap_list,
        'Price': crypto_price_list,
        'Circulating Supply': crypto_circulating_supply_list,
        'Volume (24hr)': crypto_voulume_24hr_list,
        '% 1h': crypto_pct_1hr_list,
        '% 24h': crypto_pct_24hr_list,
        '% 7d': crypto_pct_7day_list,
    })

    # Convert date format for better readability
    df['Date'] = df['Date'].apply(lambda x: datetime.strptime(x.split('/')[-2], '%Y%m%d').strftime('%Y-%m-%d'))
    df.to_csv('historical_crypto_data.csv', index=False)
    print("Data saved to 'historical_crypto_data.csv'.")

# Main execution
if __name__ == "__main__":
    scrape_date()

    start_date = datetime.strptime(scrape_date_list[0].split('/')[-2], "%Y%m%d").strftime('%Y-%m-%d')
    end_date = datetime.strptime(scrape_date_list[-1].split('/')[-2], "%Y%m%d").strftime('%Y-%m-%d')
    print(f'There are {len(scrape_date_list)} dates (Sundays) between {start_date} and {end_date}')

    for i, date in enumerate(scrape_date_list):
        scrape_data(date)
        print(f"Completed: {i + 1} out of {len(scrape_date_list)}")

    process_and_save_data()


There are 608 dates (Sundays) available for scraping from CoinMarketCap historical data.
There are 608 dates (Sundays) between 2013-04-28 and 2024-12-15
Completed: 1 out of 608
Completed: 2 out of 608
Completed: 3 out of 608
Completed: 4 out of 608
Completed: 5 out of 608
Completed: 6 out of 608
Completed: 7 out of 608
Completed: 8 out of 608
Completed: 9 out of 608
Completed: 10 out of 608
Completed: 11 out of 608
Completed: 12 out of 608
Completed: 13 out of 608
Completed: 14 out of 608
Completed: 15 out of 608
Completed: 16 out of 608
Completed: 17 out of 608
Completed: 18 out of 608
Completed: 19 out of 608
Completed: 20 out of 608
Completed: 21 out of 608
Completed: 22 out of 608
Completed: 23 out of 608
Completed: 24 out of 608
Completed: 25 out of 608
Completed: 26 out of 608
Completed: 27 out of 608
Completed: 28 out of 608
Completed: 29 out of 608
Completed: 30 out of 608
Completed: 31 out of 608
Completed: 32 out of 608
Completed: 33 out of 608
Completed: 34 out of 608
Comple

In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime

class MarketScraper:
    def __init__(self):
        #self.crypto_url = "https://coinmarketcap.com/"
        self.fiat_url = "https://www.xe.com/currencytables/"
        self.gold_url = "https://www.investing.com/commodities/gold"
        self.headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.82 Safari/537.36"
        }

    def scrape_top_cryptos(self):
        response = requests.get(self.crypto_url, headers=self.headers)
        soup = BeautifulSoup(response.content, "html.parser")

        cryptos = []

        # Scraping the top 20 cryptos from CoinMarketCap
        for row in soup.select("table.cmc-table tr")[1:21]:
            name = row.select_one(".coin-item-symbol").text.strip()
            price = row.select_one(".price___3rj7O").text.strip()
            cryptos.append({"Name": name, "Price": price})

        return cryptos

    def scrape_top_fiats(self):
        response = requests.get(self.fiat_url, headers=self.headers)
        soup = BeautifulSoup(response.content, "html.parser")

        fiats = []

        # Scraping the top 10 fiat currencies
        rows = soup.select("table.currencytables__Table-sc-14yrh3k-0 tr")[1:11]
        for row in rows:
            cells = row.select("td")
            name = cells[0].text.strip()
            rate = cells[1].text.strip()
            fiats.append({"Currency": name, "Rate (to USD)": rate})

        return fiats

    def scrape_gold_price(self):
        response = requests.get(self.gold_url, headers=self.headers)
        soup = BeautifulSoup(response.content, "html.parser")

        # Scraping gold price from Investing.com
        price = soup.select_one(".instrument-price_instrument-price__3uw25 .text-2xl").text.strip()

        return {"Gold Price (USD/oz)": price}

    def collect_data(self):
        #cryptos = self.scrape_top_cryptos()
        fiats = self.scrape_top_fiats()
        gold = self.scrape_gold_price()

        # Compile into a dictionary
        data = {
            "Cryptos": cryptos,
            "Fiats": fiats,
            "Gold": gold
        }
        return data

    def save_to_csv(self, data):
        today = datetime.datetime.now().strftime("%Y-%m-%d")

        # Save cryptos
        crypto_df = pd.DataFrame(data["Cryptos"])
        crypto_df.to_csv(f"cryptos_{today}.csv", index=False)

        # Save fiats
        fiat_df = pd.DataFrame(data["Fiats"])
        fiat_df.to_csv(f"fiats_{today}.csv", index=False)

        # Save gold price
        gold_df = pd.DataFrame([data["Gold"]])
        gold_df.to_csv(f"gold_{today}.csv", index=False)

        print("Data saved to CSV files.")

if __name__ == "__main__":
    scraper = MarketScraper()
    market_data = scraper.collect_data()
    scraper.save_to_csv(market_data)


AttributeError: 'NoneType' object has no attribute 'text'